# Apprenticeship Data Extraction 🏗️

This notebook extracts UK apprenticeship data from the [latest data for apprenticeships provided by UK.GOV](https://explore-education-statistics.service.gov.uk/find-statistics/apprenticeships).

## **What This Notebook Does**

- **Extracts** the ZIP file containing government-published apprenticeship datasets
- **Scans all CSV files** and identifies those containing **LAD-related information**
- **Moves relevant files** into a dedicated directory for analysis
- **Prepares the data** for visualization and mapping

## **Why This Is Needed**
The UK government publishes apprenticeship data in a **large ZIP archive**, containing **many different CSVs** with various indicators. **Not all files are relevant**, so this notebook helps by:

- **Filtering out unnecessary data**
- **Keeping only files with Local Authority District (LAD) information**

## Importing All Modules 📚🔍

In [1]:
import zipfile  # Module for handling ZIP files
import os  # Module for interacting with the operating system
import re  # Regular expressions for text operations
import pandas as pd  # Data manipulation and analysis
import shutil  # for copying files

## Custom File Paths 👣

In future updates from UK.GOV, this notebook will allow users to specify:

- The path to the ZIP file they want to extract;
- The folder where the extracted files should be saved.

Uncomment the lines below to set the path to the ZIP file and the folder where the extracted files should be saved.

In [2]:
# zip_path = input("Enter the path to the ZIP file: ").strip()
# extract_path = input("Enter the destination folder for extracted files: ").strip()

# print(f"ZIP file path set to: {zip_path}")
# print(f"Extraction folder set to: {extract_path}")

In [3]:
zip_path = "/work/apprenticeships_2023-24.zip"  # Path to the ZIP file
extracted_path = "/work/extracted/data_jan25/" # Destination folder for extracted files

In [4]:
os.makedirs(extracted_path, exist_ok=True) # Creates the destination folder if it doesn't exist

## Extracting Data ✨

The code below extracts the data from the ZIP file and saves it to the destination folder defined above.

In [5]:
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extracted_path)  # Extracts everything into /mnt/data/extracted/

os.listdir(extracted_path)  # Lists the extracted folders to confirm

['data-guidance', 'data', 'supporting-files']

In [6]:
# Define the correct path
data_path = extracted_path + "/data"

# List all files in the folder
file_list = os.listdir(data_path) if os.path.exists(data_path) else "Folder not found"

## Checking Extracted Data againgst Expected in Data Guidance ✅

In [7]:
# Load the data guidance file
guidance_file_path = "/work/extracted/data_jan25/data-guidance/data-guidance.txt"

# Read the file content
with open(guidance_file_path, "r") as file:
    guidance_text = file.read()

# Extract filenames using regex
csv_filenames = re.findall(r"Filename:\s*(\S+\.csv)", guidance_text)

# Print extracted filenames
for filename in csv_filenames:
    print(filename)


app-narts-learner-detailed.csv
app-narts-deprivation.csv
app-narts-provider-level-fwk-std.csv
app-narts-provider-type.csv
app-narts-subject-demographics.csv
app-narts-subject-and-level-detailed.csv
app-vacancies-adverts-2024-nov.csv
app-service-commitments-202324-nov.csv
app-duration-staylength-emplength-202324-q4.csv
fes-geography-pcon-latest-boundaries-202324-q4.csv
app-starts-since-202324-q4.csv
app-geography-pcon-lad-202324-q4.csv
app-geography-lep-eda-202324-q4.csv
app-geography-detailed-202324-q4.csv
app-geography-population-202324-q4.csv
app-latest-summary-full-year-202324-q4.csv
app-historical-summary-to-2223.csv
app-learner-deprivation-202324-q4.csv
app-learner-detailed-202324-q4.csv
app-learner-lldd-202324-q4.csv
app-monthly-starts-202324-nov.csv
app-provider-starts-202324-q4.csv
app-pubsec-summary-2024.csv
app-service-redundancies-202324-nov.csv
app-subject-standards-202324-q4.csv
app-subject-detailed-202324-q4.csv
app-subject-levy-202324-q4.csv
app-subject-demographics-2023

In [8]:
# Compare expected filenames with actual extracted files using partial matches
for filename in csv_filenames:
    match_found = any(actual_file.startswith(filename.split(".csv")[0]) for actual_file in file_list)
    
    if match_found:
        print(f"✅ {filename}")  # Green tick if a match is found
    else:
        print(f"❌ {filename}")  # Red cross if no match is found


✅ app-narts-learner-detailed.csv
✅ app-narts-deprivation.csv
✅ app-narts-provider-level-fwk-std.csv
✅ app-narts-provider-type.csv
✅ app-narts-subject-demographics.csv
✅ app-narts-subject-and-level-detailed.csv
✅ app-vacancies-adverts-2024-nov.csv
✅ app-service-commitments-202324-nov.csv
✅ app-duration-staylength-emplength-202324-q4.csv
✅ fes-geography-pcon-latest-boundaries-202324-q4.csv
✅ app-starts-since-202324-q4.csv
✅ app-geography-pcon-lad-202324-q4.csv
✅ app-geography-lep-eda-202324-q4.csv
✅ app-geography-detailed-202324-q4.csv
✅ app-geography-population-202324-q4.csv
✅ app-latest-summary-full-year-202324-q4.csv
✅ app-historical-summary-to-2223.csv
✅ app-learner-deprivation-202324-q4.csv
✅ app-learner-detailed-202324-q4.csv
✅ app-learner-lldd-202324-q4.csv
✅ app-monthly-starts-202324-nov.csv
✅ app-provider-starts-202324-q4.csv
✅ app-pubsec-summary-2024.csv
✅ app-service-redundancies-202324-nov.csv
✅ app-subject-standards-202324-q4.csv
✅ app-subject-detailed-202324-q4.csv
✅ app-su

### Filtering and Moving LAD-Related CSV Files 📂 

This cell scans **all extracted CSV files** in `extracted_path` and:

- **Checks if they contain LAD-related columns** (`lad_name`, `lad_code`, or `Local_Authority`).
- **Moves the valid files** to `/work/LADS_data/`.
- **Skips and prints files that do not contain LAD information**.

✅ If a file contains LAD data, it is moved 
⚠️ If it does not, it remains in the original folder and is skipped


In [9]:
# Define the path
target_folder = "/work/LADS_data/"

# Ensure the target folder exists
os.makedirs(target_folder, exist_ok=True)

# List all CSV files in the extracted_path folder
csv_files = [f for f in os.listdir(extracted_path) if f.endswith(".csv")]

# Define possible LAD-related column names
lad_column_variants = ["lad_name", "lad_code", "Local_Authority"]

# Iterate through CSVs and move ones that contain any LAD-related column
for file in csv_files:
    file_path = os.path.join(extracted_path, file)
    
    try:
        df = pd.read_csv(file_path, nrows=5)  # Read first 5 rows for efficiency
        found_columns = [col for col in df.columns if any(lad in col for lad in lad_column_variants)]
        
        if found_columns:  # If any LAD-related column is found
            shutil.move(file_path, os.path.join(target_folder, file))
            print(f"✅ Moved: {file} (LAD columns found: {found_columns})")
        else:
            print(f"⚠️ Skipped: {file} (No LAD-related columns)")
    except Exception as e:
        print(f"⚠️ Error processing {file}: {e}")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=cfb480dc-687e-4482-b857-be10c729aa58' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>